In [2]:
from catboost.datasets import titanic
from pathlib import Path
import numpy as np
import os
while "freqtrade" not in os.listdir():
    os.chdir("..")

train_df, test_df = titanic()
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [8]:
null_value_stats = train_df.isnull().sum(axis=0)
null_value_stats

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

In [10]:
X = train_df.drop('Survived', axis=1)
y = train_df.Survived

In [12]:
print(X.dtypes)
categorical_features_indices = np.where(X.dtypes != float)[0]

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)
X_test = test_df

In [14]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

In [25]:
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent',
    task_type="GPU",
)

In [ ]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [17]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

KeyboardInterrupt: 

In [41]:
# Use XGBoost to train Iris model
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
import xgboost as xgb
import numpy as np

# Start training
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param = {
    'max_depth': 1,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 0,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3,
    'tree_method': 'gpu_hist',
}  # the number of classes that exist in this datset

num_round = 20000 # the number of training iterations

bst = xgb.train(param, dtrain, num_round)

preds = bst.predict(dtest)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

[16:17:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:17:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Precision = 1.0
Recall = 1.0
Accuracy = 1.0


In [8]:
from distributed import Client
os.environ["MODIN_ENGINE"] = "dask"
client = Client(n_workers=6)
import modin.pandas as mpd

In [27]:
path_data = Path.cwd().parent / "mount" / "data" / "binance" / "BTC_USDT-1m.json"
headers = ["date", "open", "high", "low", "close", "volume"]
d1 = pd.read_json(path_data)
d1.columns = headers
d1["date"] = pd.to_datetime(d1["date"], unit='ms', utc=True, infer_datetime_format=True)
d1 = d1.astype(
    dtype={'open': 'float', 'high': 'float', 'low': 'float', 'close': 'float', 'volume': 'float'}
)
d1

distributed.utils_perf - WARNING - full garbage collections took 41% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 41% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 42% CPU time recently (threshold: 10%)


,date,open,high,low,close,volume
0,2017-08-17 04:00:00+00:00,4261.48,4261.48,4261.48,4261.48,1.775183
1,2017-08-17 04:01:00+00:00,4261.48,4261.48,4261.48,4261.48,0.000000
2,2017-08-17 04:02:00+00:00,4280.56,4280.56,4280.56,4280.56,0.261074
3,2017-08-17 04:03:00+00:00,4261.48,4261.48,4261.48,4261.48,0.012008
4,2017-08-17 04:04:00+00:00,4261.48,4261.48,4261.48,4261.48,0.140796
...,...,...,...,...,...,...
2107051,2021-08-25 04:53:00+00:00,48051.01,48052.79,48024.22,48028.39,13.875819
2107052,2021-08-25 04:54:00+00:00,48028.39,48038.03,47967.93,47978.98,26.209319
2107053,2021-08-25 04:55:00+00:00,47978.98,48002.00,47933.34,47949.24,19.675199
2107054,2021-08-25 04:56:00+00:00,47949.24,47949.24,47911.34,47924.44,26.835354


In [29]:
from freqtrade.ml.loader import clean_ohlcv_dataframe
d1 = clean_ohlcv_dataframe(d1, "1m", fill_missing=True, drop_incomplete=True)
d1

KeyError: 'The grouper name date is not found'

In [51]:
import pandas as pd
df = pd.DataFrame({
    "no": [5, 8, 1],
    "ml_1": [0, 0, 1],
    "ml_2": [1, 0.5, 0],
    "buy": [0,0,0]
})
df["argmax_ml"] = df[["ml_1", "ml_2"]].idxmax(axis=1)
df.loc[df["argmax_ml"] == "ml_1", "buy"] = 1
df = df[["ml_1", "ml_2"]]
df

,ml_1,ml_2
0,0,1.0
1,0,0.5
2,1,0.0


In [15]:
# TODO: Store a list of row indexes that has NaN
# Then predict using non NaN rows
# Then concat predictions to non NaN indexes
import numpy as np
import pandas as pd

df = pd.DataFrame({
    "price1": [2, 4.1, 0.3, np.nan, 9],
    "price2": [3, 0.2, np.nan, 8, 99]
})

nan_indexes = df[df.isnull().any(axis=1)].index
    non_nan_indexes = df[~df.isnull().any(axis=1)].index
df.loc[non_nan_indexes]

,price1,price2
0,2.0,3.0
1,4.1,0.2
4,9.0,99.0


In [18]:
preds = np.array([[4.0], [3.0], [4.0]])
preds

array([[4.],
       [3.],
       [4.]])

In [19]:
# Assuming orders preserved, we can safely concat preds to non nan indexes
# (Len of preds == Len non nan rows)
df_preds = pd.DataFrame(preds)
df_preds.index = non_nan_indexes
df_preds

,0
0,4.0
1,3.0
4,4.0


In [20]:
df_with_preds = pd.concat([df, df_preds], axis=1)
df_with_preds

,price1,price2,0
0,2.0,3.0,4.0
1,4.1,0.2,3.0
2,0.3,NaN,NaN
3,NaN,8.0,NaN
4,9.0,99.0,4.0


In [21]:
df_with_preds.describe()

,price1,price2,0
count,4.000000,4.000000,3.000000
mean,3.850000,27.550000,3.666667
std,3.768731,47.742469,0.577350
min,0.300000,0.200000,3.000000
25%,1.575000,2.300000,3.500000
50%,3.050000,5.500000,4.000000
75%,5.325000,30.750000,4.000000
max,9.000000,99.000000,4.000000


In [1]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

predictions = model(x_train[:1]).numpy()
predictions

array([[-0.5586384 ,  0.5700961 ,  0.5081506 , -0.3777456 , -0.1562815 ,
        -0.48242852, -0.0175143 ,  0.14745747,  0.5195229 ,  0.16306321]],
      dtype=float32)

In [27]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 751us/step - loss: 0.2976 - accuracy: 0.9126
Epoch 2/5
1875/1875 [==============================] - 1s 745us/step - loss: 0.1430 - accuracy: 0.9570
Epoch 3/5
1875/1875 [==============================] - 1s 788us/step - loss: 0.1065 - accuracy: 0.9671
Epoch 4/5
1875/1875 [==============================] - 2s 844us/step - loss: 0.0880 - accuracy: 0.9725
Epoch 5/5
1875/1875 [==============================] - 2s 802us/step - loss: 0.0742 - accuracy: 0.9767


In [2]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]